# Blog_SQLite

We will work with the `blog.sqlite` database is available at this URL:  
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite`

## Imports


In [1]:
import sqlite3
import pandas as pd

## 1. Schema ERD

<img src=https://wagon-public-datasets.s3.amazonaws.com/01-Python/04-SQL-Advanced/blog_db_schema.png width=500>

## Creating the connection and cursor

In [2]:
connection = sqlite3.connect("data/blog.sqlite")
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

## Finding the tables in the database

In [5]:
query = """
        SELECT name FROM sqlite_master  
        WHERE type='table';
        """
cursor.execute(query)
rows = cursor.fetchall()

print(f"""
{type(rows) = }
{len(rows) = }
-----------------------------------------
{[row[row.keys()[0]] for row in rows] = }
""")



type(rows) = <class 'list'>
len(rows) = 4
-----------------------------------------
[row[row.keys()[0]] for row in rows] = ['sqlite_sequence', 'users', 'posts', 'likes']



## 2. Most liked posts

Getting **the 3 most liked posts**:

In [11]:
query = """
        SELECT
            posts.id,
            posts.title,
            COUNT(*) like_count
        FROM posts
        JOIN likes ON posts.id = likes.post_id
        GROUP BY posts.id
        ORDER BY like_count DESC
        LIMIT 3
        """
cursor.execute(query)
rows = cursor.fetchall()

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
------------------------------------------------------------------------------------
""")

pd.read_sql_query(query, connection)


------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 3
------------------------------------------------------------------------------------



,id,title,like_count
0,143,Half imagine another.,84
1,83,Side foot leader popular.,82
2,99,Area paper whatever mean.,81


## 3. Find the three users who 'liked' the most

In [12]:
query = """
        SELECT
            users.id,
            users.first_name || " " || users.last_name full_name,
            COUNT(*) like_count
        FROM users
        JOIN likes ON users.id = likes.user_id
        GROUP BY users.id 
        ORDER BY like_count DESC
        LIMIT 3
        """
cursor.execute(query)
rows = cursor.fetchall()

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
------------------------------------------------------------------------------------
""")

pd.read_sql_query(query, connection)


------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 3
------------------------------------------------------------------------------------



,id,full_name,like_count
0,43,Michael Allen,236
1,12,Donna Ramirez,233
2,44,Hayley Williams,227


## 4. Find the most liked author

In [15]:
query = """
        SELECT
            users.id,
            users.first_name || " " || users.last_name full_name,
            COUNT(*) like_count
        FROM users
        JOIN posts ON users.id = posts.user_id 
        JOIN likes ON posts.id = likes.post_id
        GROUP BY users.id 
        ORDER BY like_count DESC 
        LIMIT 1
        """
cursor.execute(query)
rows = cursor.fetchall()

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
------------------------------------------------------------------------------------
""")

pd.read_sql_query(query, connection)


------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 1
------------------------------------------------------------------------------------



,id,full_name,like_count
0,57,Teresa Moore,647


## 5. Who are the authors of the 3 most liked posts?

In [22]:
query = """
        SELECT
            users.id,
            users.first_name || " " || users.last_name full_name,
            COUNT(*) like_count
        FROM users
        JOIN posts ON users.id = posts.user_id 
        JOIN likes ON posts.id = likes.post_id
        GROUP BY posts.id 
        ORDER BY like_count DESC 
        LIMIT 3
        """
cursor.execute(query)
rows = cursor.fetchall()

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
------------------------------------------------------------------------------------
""")

pd.read_sql_query(query, connection)


------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 3
------------------------------------------------------------------------------------



,id,full_name,like_count
0,72,Melissa Henry,84
1,63,Cynthia Raymond,82
2,64,Alexander Cook,81


## 6. How many people liked at least one post?

In [23]:
query = """
        SELECT
            COUNT(DISTINCT user_id) Liker_Count
        FROM likes
       """
cursor.execute(query)
rows = cursor.fetchall()

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
------------------------------------------------------------------------------------
""")

pd.read_sql_query(query, connection)



------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 1
------------------------------------------------------------------------------------



,Liker_Count
0,49


## 7. Compute the cumulative number of likes per day

In [24]:
query = """
        SELECT
            created_at AS date,
            COUNT(*) daily_like_count,
            SUM(COUNT(*)) OVER(ORDER BY created_at) cumulative_daily_like_count
        FROM likes
        GROUP BY date
        """
cursor.execute(query)
rows = cursor.fetchall()

df = pd.read_sql_query(query, connection)

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
{df.keys()}
------------------------------------------------------------------------------------
""")
df


------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 336
Index(['date', 'daily_like_count', 'cumulative_daily_like_count'], dtype='object')
------------------------------------------------------------------------------------



,date,daily_like_count,cumulative_daily_like_count
0,2019-01-01,24,24
1,2019-01-02,34,58
2,2019-01-03,40,98
3,2019-01-04,36,134
4,2019-01-05,27,161
...,...,...,...
331,2019-12-24,35,9882
332,2019-12-25,33,9915
333,2019-12-26,32,9947
334,2019-12-27,23,9970


## 8. Who's the biggest fan/ fans of each author?

The biggest fan/ fans of an author is defined as the user or users who liked the author's posts the most. i.e. if there is a tie between fans that both liked an author 20 times, both fans should be returned alongside their like count and the author in question.

In [25]:
query = """
        WITH liker_per_author AS (
                                    SELECT
                                        posts.user_id author_id,
                                        likes.user_id liker_id,
                                        COUNT(*) like_count
                                    FROM likes
                                    JOIN posts ON likes.post_id = posts.id
                                    GROUP BY author_id, liker_id
                                ),
                        ranks AS (
                                    SELECT
                                        authors.first_name || " " || authors.last_name author_full_name,
                                        likers.first_name || " " || likers.last_name biggest_fan_full_name,
                                        like_count,
                                        RANK() OVER(PARTITION BY author_id ORDER BY like_count DESC) rank
                                    FROM liker_per_author
                                    JOIN users authors ON liker_per_author.author_id = authors.id
                                    JOIN users likers ON liker_per_author.liker_id = likers.id
                                    ORDER BY author_full_name
                                )
        SELECT 
            author_full_name, 
            biggest_fan_full_name, like_count 
        FROM ranks
        WHERE rank = 1
        """
cursor.execute(query)
rows = cursor.fetchall()

df = pd.read_sql_query(query, connection)

print(f"""
------------------------------------------------------------------------------------
{type(rows) = }
{len(rows) = }
{df.keys()}
------------------------------------------------------------------------------------
""")
df



------------------------------------------------------------------------------------
type(rows) = <class 'list'>
len(rows) = 40
Index(['author_full_name', 'biggest_fan_full_name', 'like_count'], dtype='object')
------------------------------------------------------------------------------------



,author_full_name,biggest_fan_full_name,like_count
0,Alexander Cook,Jeremiah Ruiz,20
1,Brenda Griffin,Michael Allen,12
2,Brett Johnston,Tristan Kelley,19
3,Brett Johnston,Gary Garza,19
4,Brett Johnston,Kayla Higgins,19
5,Brittany Miller,Barbara Hurst,16
6,Brittany Miller,Gary Garza,16
7,Cynthia Raymond,Donald Gray,24
8,David Boyd,Barbara Hurst,18
9,Donna Smith,Karen Brown,20


## Creating and exporting  csv to work in Power BI (All tables in the database)


In [28]:
query_exploring = """
                SELECT name FROM sqlite_master  
                WHERE type='table';
                """

cursor.execute(query_exploring)
rows = cursor.fetchall()
print(rows)

for index, row in enumerate(rows[1::]):
    print(f"""{index}_{row[0]}""")
    query_table = f"""
            SELECT * FROM {row[0]}  
            """
    print(query_table)
    movies = pd.read_sql(query_table, connection)
    movies.to_csv(f"data/csv_blog_{row[0]}.csv", index=False)

[<sqlite3.Row object at 0x7fa91a1a6170>, <sqlite3.Row object at 0x7fa91a1a6050>, <sqlite3.Row object at 0x7fa91a1a5b40>, <sqlite3.Row object at 0x7fa91a1a5a20>]
0_users

            SELECT * FROM users  
            
1_posts

            SELECT * FROM posts  
            
2_likes

            SELECT * FROM likes  
            
